In [0]:
!pip3 install torch torchvision

In [0]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([transforms.Resize((32,32)),
                               #transforms.Grayscale(),
                               transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=1),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5,))
                               ])

training_dataset = datasets.SVHN(root='./data', download=True, split='train', transform=transform)
validation_dataset = datasets.SVHN(root='./data', download=True, split='test', transform=transform)

training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=100, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size =100, shuffle=False)

def convert_image(tensor):
  image = tensor.cpu().clone().detach().numpy()
  image = image.transpose(1, 2, 0)
  image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
  image = image.clip(0, 1)
  return image

In [0]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 60, 5, 1)
        self.conv2 = nn.Conv2d(60, 150, 5, 1)

        self.fc1 = nn.Linear(5*5*150, 500)

        self.dropout1 = nn.Dropout(0.5)
        
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)

        x = x.view(-1, 5*5*150)
        x = F.relu(self.fc1(x))

        x = self.dropout1(x)

        x = self.fc2(x)
        return x

In [0]:
model = LeNet().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)
epochs = 150

training_loss_history = []
training_corrects_history = []
validation_loss_history = []
validation_corrects_history = []

for epoch in range(epochs):
  
  training_loss = 0.0
  training_correct = 0.0
  validation_loss = 0.0
  validation_correct = 0.0
  
  for inputs, labels in training_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    loss = loss_func(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    _, preds = torch.max(outputs, 1)
    training_loss += loss.item()
    training_correct += torch.sum(preds == labels.data)

  with torch.no_grad():
    for val_inputs, val_labels in validation_loader:
      val_inputs = val_inputs.to(device)
      val_labels = val_labels.to(device)
      val_outputs = model(val_inputs)
      val_loss = loss_func(val_outputs, val_labels)
      
      _, val_preds = torch.max(val_outputs, 1)
      validation_loss += val_loss.item()
      validation_correct += torch.sum(val_preds == val_labels.data)
      
    epoch_loss = training_loss/len(training_loader)
    epoch_acc = training_correct.float()/ len(training_loader)

    training_loss_history.append(epoch_loss)
    training_corrects_history.append(epoch_acc)
    
    val_epoch_loss = validation_loss/len(validation_loader)
    val_epoch_acc = validation_correct.float()/ len(validation_loader)
    validation_loss_history.append(val_epoch_loss)
    validation_corrects_history.append(val_epoch_acc)

In [0]:
plt.plot(training_loss_history, label='training loss')
plt.plot(validation_loss_history, label='validation loss')
plt.legend()

In [0]:
plt.plot(training_corrects_history, label='training accuracy')
plt.plot(validation_corrects_history, label='validation accuracy')
plt.legend()

In [0]:
Iterator = iter(validation_loader)
images, labels = dataiter.next()
images = images.to(device)
labels = labels.to(device)
output = model(images)
_, preds = torch.max(output, 1)

fig = plt.figure(figsize=(30, 4))

for ndx in np.arange(20):
  img = fig.add_subplot(2, 10, ndx+1, xticks=[], yticks=[])
  plt.imshow(im_convert(images[ndx]))
  img.set_title("{} ({})".format(str(preds[ndx].item()), str(labels[ndx].item())), color=("green" if preds[ndx]==labels[ndx] else "red"))

In [0]:
torch.save(model.state_dict(), "digit_classification_cnn.pt")

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'digit_classification_cnn.pt'})
uploaded.SetContentFile('digit_classification_cnn.pt')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))